# NBA 2021 Project
### Francis Peng, Annie Seo

# Introduction and Motivation

- Contract/value/net-worth
- Shot selection, 2s, 3s, midrange, layup, etc.
- Predicting outcomes/win percentage based on midgame stats of indiivudal players.
- ## Value of the midrange shot?

# Data and Code Setup

We will use the following API which scrapes NBA basketball statistics from basketball-reference.com
https://github.com/vishaalagartha/basketball_reference_scraper

In [1]:
# !pip install basketball-reference-scraper
# !pip install unidecode

In [2]:
# Boilerplate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import scipy.stats as stats
from scipy.optimize import minimize

In [3]:
# Load basketball-reference libraries
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
from basketball_reference_scraper.players import get_stats, get_game_logs, get_player_headshot
from basketball_reference_scraper.seasons import get_schedule, get_standings
from basketball_reference_scraper.box_scores import get_box_scores
from basketball_reference_scraper.pbp import get_pbp
from basketball_reference_scraper.shot_charts import get_shot_chart

## First, we retrieve the players with the best midrange efficiency. To filter out anomalies, we find players with relatively high midrange volume (top 50%). 

In [36]:
team_abbr = {
    'ATLANTA HAWKS' : 'ATL',
'BOSTON CELTICS' : 'BOS',
'BROOKLYN NETS' : 'BRK',
'CHICAGO BULLS' : 'CHI',
'CHARLOTTE HORNETS' : 'CHO',
'CLEVELAND CAVALIERS' : 'CLE',
'DALLAS MAVERICKS' : 'DAL',
'DENVER NUGGETS' : 'DEN',
'DETROIT PISTONS' : 'DET',
'GOLDEN STATE WARRIORS' : 'GSW',
'HOUSTON ROCKETS' : 'HOU',
'INDIANA PACERS' : 'IND',
'LOS ANGELES CLIPPERS' : 'LAC',
'LOS ANGELES LAKERS' : 'LAL',
'MEMPHIS GRIZZLIES' : 'MEM',
'MIAMI HEAT' : 'MIA',
'MILWAUKEE BUCKS' : 'MIL',
'MINNESOTA TIMBERWOLVES' : 'MIN',
'NEW ORLEANS PELICANS' : 'NOP',
'NEW YORK KNICKS' : 'NYK',
'OKLAHOMA CITY THUNDER' : 'OKC',
'ORLANDO MAGIC' : 'ORL',
'PHILADELPHIA 76ERS' : 'PHI',
'PHOENIX SUNS' : 'PHO',
'PORTLAND TRAIL BLAZERS' : 'POR',
'SACRAMENTO KINGS' : 'SAC',
'SAN ANTONIO SPURS' : 'SAS',
'TORONTO RAPTORS' : 'TOR',
'UTAH JAZZ' : 'UTA',
'WASHINGTON WIZARDS' : 'WAS'
}
team_abbr_df = pd.DataFrame.from_dict(team_abbr, orient='index')
team_abbr_df

,0
ATLANTA HAWKS,ATL
BOSTON CELTICS,BOS
BROOKLYN NETS,BRK
CHICAGO BULLS,CHI
CHARLOTTE HORNETS,CHO
CLEVELAND CAVALIERS,CLE
DALLAS MAVERICKS,DAL
DENVER NUGGETS,DEN
DETROIT PISTONS,DET
GOLDEN STATE WARRIORS,GSW


In [37]:
# Retrieve Schedule
schedule_2021 = get_schedule(2021, playoffs=False)

In [38]:
schedule_2021 = schedule_2021[['DATE', 'VISITOR', 'HOME']]
schedule_2021 = schedule_2021.rename(columns={'VISITOR' : 'visit', 'HOME' : 'home'})
schedule_2021['visit'] = schedule_2021['visit'].str.upper()
schedule_2021['home'] = schedule_2021['home'].str.upper()
schedule_2021.join(team_abbr_df, on='visit', how='left', lsuffix='_sch', rsuffix='_vst')

,DATE,visit,home,0
0,2020-12-22,GOLDEN STATE WARRIORS,BROOKLYN NETS,GSW
1,2020-12-22,LOS ANGELES CLIPPERS,LOS ANGELES LAKERS,LAC
2,2020-12-23,CHARLOTTE HORNETS,CLEVELAND CAVALIERS,CHO
3,2020-12-23,NEW YORK KNICKS,INDIANA PACERS,NYK
4,2020-12-23,MIAMI HEAT,ORLANDO MAGIC,MIA
...,...,...,...,...
1158,2021-06-26,PHOENIX SUNS,LOS ANGELES CLIPPERS,PHO
1159,2021-06-27,MILWAUKEE BUCKS,ATLANTA HAWKS,MIL
1160,2021-06-28,LOS ANGELES CLIPPERS,PHOENIX SUNS,LAC
1161,2021-06-29,MILWAUKEE BUCKS,ATLANTA HAWKS,MIL


In [33]:
schedule_2021['VISITOR'] = team_abbr.get(schedule_2021['visit'])

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [24]:
get_shot_chart('2020-12-22', 'GSW','BRK')

{'GSW':           x        y  QUARTER TIME_REMAINING          PLAYER MAKE_MISS  VALUE  \
 0   24.9 ft  3.08 ft        1        11:11.0   James Wiseman      MAKE      2   
 1   47.4 ft  7.86 ft        1        10:31.0  Andrew Wiggins      MISS      3   
 2   11.8 ft  27.1 ft        1        10:16.0   Stephen Curry      MISS      3   
 3   47.0 ft  10.7 ft        1         9:57.0  Andrew Wiggins      MISS      3   
 4   26.8 ft  5.87 ft        1         9:38.0   Stephen Curry      MAKE      2   
 ..      ...      ...      ...            ...             ...       ...    ...   
 92  26.7 ft  26.1 ft        4         1:54.0   James Wiseman      MAKE      2   
 93   9.8 ft  8.66 ft        4         1:30.0   James Wiseman      MISS      2   
 94  1.40 ft  14.3 ft        4         1:08.0    Jordan Poole      MISS      3   
 95  26.0 ft  11.5 ft        4         0:10.0   James Wiseman      MISS      2   
 96  14.2 ft  27.0 ft        4         0:07.0   Mychal Mulder      MAKE      3   
 
    DIS

## Next, we compare their midrange efficiency against their three-point efficiency, again taking volume into consideration.

# Methodology and Analysis

# Conclusion